In [ ]:
!pip install langchain-anthropic langgraph-checkpoint-sqlite

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [3]:
from langchain_anthropic import ChatAnthropic

personal_brand_strategy_builder_llm = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0.5,
    max_tokens=1024,
)

content_creator_llm = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0.5,
    max_tokens=1024,
)


In [4]:
import operator
from typing import List
from typing_extensions import TypedDict, Annotated
from pydantic import BaseModel, Field

class PersonalBrandStrategyBuilder(BaseModel):
    strategy: str = Field(default="", description="The strategy to use for the personal brand")
    target_audience: str = Field(default="", description="The target audience for the personal brand")
    key_themes: str = Field(default="", description="The key themes for the personal brand")
    topics: str = Field(default="", description="The topics for the personal brand")
    tone: str = Field(default="", description="The tone for the personal brand")
    voice: str = Field(default="", description="The voice for the personal brand")
    personal_goals: str = Field(default="", description="The personal goals for the personal brand")

class Ideas(BaseModel):
    idea: str = Field(default = "", description="The idea for the post")
    links: List[str] = Field(default = [], description="The links for the post")

class ContentCreator(BaseModel):
    post: str = Field(default = "", description="The post for the personal brand")

class State(TypedDict):
    personal_brand_strategy: PersonalBrandStrategyBuilder
    posts: Annotated[List[ContentCreator], operator.add]
    human_feedback: str


In [5]:
def dummy_node(state: State):
    pass


In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

personal_brand_strategy_instructions = """
You are tasked with creating a comprehensive strategy for my personal brand. Follow these instructions carefully:

1. Review the personal goals of the personal brand:
{personal_goals}

2. Examine any human feedback that has been optionally provided to guide the creation of the personal brand:
{human_feedback}

3. Create a strategy that will help me achieve my personal goals.

4. Define the target audience for the personal brand.

5. Establish key themes and topics for the personal brand.

6. Define the tone and voice of the personal brand.
"""

def create_personal_brand_strategy(state: State):
    human_feedback = state['human_feedback']
    personal_goals = state['personal_brand_strategy'].personal_goals

    structured_llm = personal_brand_strategy_builder_llm.with_structured_output(PersonalBrandStrategyBuilder)

    system_message = personal_brand_strategy_instructions.format(personal_goals=personal_goals, human_feedback=human_feedback)

    personal_brand_strategy = structured_llm.invoke(
        [SystemMessage(content=system_message)] + [HumanMessage(content="Generate a personal brand strategy based on the provided information.")]
    )
    personal_brand_strategy.personal_goals = personal_goals

    print("Personal Brand Strategy: ", personal_brand_strategy)

    return {
        'personal_brand_strategy': personal_brand_strategy
    }


In [7]:
content_creator_instructions = """
You are tasked with creating several tweets based on a given personal brand strategy and content ideas. The number of posts could vary depending on the number of ideas provided. Follow these instructions carefully:

1. Personal brand strategy:
{strategy}

2. Examine any human feedback that has been optionally provided to guide the creation of the posts:
{human_feedback}

3. Target audience:
{audience}

4. Key themes:
{key_themes}

5. Tone:
{tone}

6. Voice:
{voice}

7. Personal objectives:
{personal_objectives}
"""

def create_content_creator(state: State):
    strategy = state['personal_brand_strategy'].strategy
    audience = state['personal_brand_strategy'].target_audience
    key_themes = state['personal_brand_strategy'].key_themes
    tone = state['personal_brand_strategy'].tone
    voice = state['personal_brand_strategy'].voice
    personal_objectives = state['personal_brand_strategy'].personal_goals
    human_feedback = state['human_feedback']

    print("Human Feedback: ", human_feedback)

    structured_llm = content_creator_llm.with_structured_output(ContentCreator)

    system_message = content_creator_instructions.format(strategy=strategy, audience=audience, key_themes=key_themes, tone=tone, voice=voice, personal_objectives=personal_objectives, human_feedback=human_feedback)

    content_creator = structured_llm.invoke(
        [SystemMessage(content=system_message)] + [HumanMessage(content="Create a tweet based on the provided information.")]
    )

    posts = state['posts']
    posts.append(content_creator)

    return {
        'posts': posts
    }

In [8]:
def repeat_task(state: State):
    human_feedback = state.get("human_feedback", "")
    any_feedback = bool(human_feedback)

    print("Human Feedback: ", human_feedback)
    print("Human Feedback, falsy or truthy: ", any_feedback)

    return any_feedback


In [ ]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(State)

graph_builder.add_node("create_personal_brand_strategy", create_personal_brand_strategy)
graph_builder.add_node("create_content", create_content_creator)
graph_builder.add_node("dummy_create_personal_brand_strategy", dummy_node)
graph_builder.add_node("dummy_create_content", dummy_node)

graph_builder.add_conditional_edges("dummy_create_personal_brand_strategy", repeat_task, { True: "create_personal_brand_strategy", False: "create_content" })
graph_builder.add_conditional_edges("dummy_create_content", repeat_task, { True: "create_content", False: END })

graph_builder.add_edge(START, "create_personal_brand_strategy")
graph_builder.add_edge("create_personal_brand_strategy", "dummy_create_personal_brand_strategy")
graph_builder.add_edge("create_content", "dummy_create_content")


In [10]:
personal_brand_strategy = PersonalBrandStrategyBuilder()

personal_brand_strategy.personal_goals = "I am an engineer who wants to develop my personal brand. I have worked in several roles including Data Architect, Cloud Architect and CloudAdministrator. I want to teach others to develop better their careers. We will focus on creating tweets without images nor videos."

initial_state = State(posts=[], personal_brand_strategy=personal_brand_strategy, human_feedback="")


In [11]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [12]:
thread = { "configurable": { "thread_id": "1" } }

graph = graph_builder.compile(checkpointer=memory, interrupt_before=["dummy_create_personal_brand_strategy", "dummy_create_content"])


In [ ]:
import sys

num_interruptions = 3
i = 0
state_to_send = initial_state

while i < num_interruptions:
    for event in graph.stream(state_to_send, thread):
        print(event)

    state_to_send = None
    sys.stdout.flush()

    human_feedback = input("Any Feedback?: ")
    graph.update_state(thread, { "human_feedback": human_feedback })

    if bool(human_feedback):
        num_interruptions += 1

    i += 1

In [ ]:
from IPython.display import Image

display(Image(graph.get_graph().draw_mermaid_png()))